# Non-Gaussian Measurements Tutorial 2

## Overview

There are many, many ways in which non-Gaussian / ambiguous measurements occur, this tutorial will simply pick one common example to illustrate the basic concepts.  We ground the tutorial in an example: imagine a wheeled robot travelling along in a straight line, using wheel encoders to estimate the distance travelled.  There is, however, a quirk, from time to time the robot gets stuck on something and the wheel's spin for a few rotations before progress continues.  We assume that our software is able to detect when the wheels are slipping, but the process is only 40% accurate.  In this tutorial we will build a basic robot localization process that can handle \"bad\" measurement data.  We restrict this to a one dimensional example with only a few variables and factors, to help familiarize yourself with multi-modal beliefs.  

This example shows [one of four](https://juliarobotics.org/Caesar.jl/latest/concepts/why_nongaussian/) mechanisms by which non-Gaussian behavior can get introduced into a factor graph solution, see other tutorials for other mechanisms.  We will build a factor graph that contains multi-modal belief as well as `Uniform` and `Rayleigh` distributions, to help familiarize non-Gaussian measurements.   The ambiguous measurement example shown in this tutorial can readily be incorporated in other use cases, and also illustrates how consensus can occur when more information is included -- i.e. reducing uncertainties to unimodal marginal beliefs on each variable.

This tutorial introduces non-Gaussian behavior through non-Gaussian measurements in the factor graph.  This tutorial shows how measurements do not have to follow a unimodal bell curve (i.e. normal/Gaussian), but can instead introduce non-Gaussian beliefs and let the joint inference find the best marginal posterior estimates on each of the desired variables in the factor graph.  The tutorial shows a multi-modal uncertainty can be introduced from non-Gaussian measurements, and then transmitted through the system.  


### Signatures Used

`ContinuousScalar`, `Prior`, `LinearRelative`, `Mixture`, `Normal`

## Ambiguous Data Example

To start, the only required package is `NavAbilitySKD`.

In [2]:
# If not installed already, the NavAbilitySDK can be installed with:
pip install navabilitysdk

     |████████████████████████████████| 1.5 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 5.4 MB/s eta 0:00:011
     |████████████████████████████████| 64 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 86 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 16.8 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 280 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 189 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 308 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 4.6 MB/s eta 0:00:01     |███████████████████████████▊    | 1.3 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 102 kB 5.2 MB

In [3]:
from navability.entities import *
from navability.services import *
from uuid import uuid4
import asyncio

 
## Starting a 1D Factor Graph

### Variable, `ContinuousScalar`

The first thing to do is setup a client-context to talk with the NavAbility Platform.
The next step is to describe the inference problem with a graphical model by populating the factor graph with variable nodes.
The variable nodes are identified by `String`s, namely `x0, x1, x2, x3`.

In [7]:
navability_client = NavAbilityHttpsClient()
# you need a unique userId:robotId, and can keep using that across all tutorials
userId = "Guest"
robotId = "SDKpy_"+(str(uuid4())[0:3])

sessionId = "Tutorial2_"+(str(uuid4())[0:3])
client = Client("Guest", robotId, sessionId)


In [9]:
# add the first node
v = Variable("x0", VariableType.ContinuousScalar.value)

result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")

# Wait for it to be loaded.
await waitForCompletion(navability_client, [result_id])

Added x0 with result ID 01G0029N6NA5DNHM29JHACE970


### Prior Factor, (Euclidean(1))

Factor graphs are bipartite graphs with `factors` that act as mathematical structure between interacting `variables`.
After adding node `x0`, a singleton factor of type `Prior` (which was defined by the user earlier) is 'connected to' variable node `x0`.
This unary factor is taken as a `Normal` distribution with zero mean and a standard deviation of `1`.


In [10]:
# this is unary (prior) factor and does not immediately trigger autoinit of :x0.
prior = Factor("x0f1", "Prior", ["x0"],
      FactorData(
          fnc=Prior(
              Z=Normal(0, 1)
          ).dump()  # This is a generator for a Prior
      ))
# REF: addFactor!(fg, [:x0], Prior(Normal(0,1)))
result_id = await addFactor(navability_client, client, prior)

**Note**, this example is using just `Prior`, which is a simplification for the sake of this tutorial.  Look out for on-manifold prior factors in other code examples.

### Visualizing Graph Topology

In [11]:
# As with tutorials 1 we can visualize the factor graph in the NavAbility App as we add variables.
print("https://app.navability.io/cloud/graph/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

https://app.navability.io/cloud/graph/?userId=Guest&robotStartsWith=SDKpy_0bd&sessionStartsWith=Tutorial2_678


The topological graph plot obtained from the NavAbility App above shows the two node factor graph, one variable and one prior factor.  This example uses graph-based automatic variable initialization which was discussed in ICRA 2022 Tutorial 1.

### Next Variable and Relative Factor

Now let's add a second variable `x1`, and connect it to `x0` with a `LinearRelative` factor.

In [12]:
# let's wait to make sure all nodes were added
await waitForCompletion(navability_client, [result_id])

# add x1
v = Variable("x1", "IncrementalInference.ContinuousScalar") #VariableType.ContinuousScalar.value)
result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")

await getStatusLatest(navability_client, result_id)


# P(Z | :x1 - :x0 ) where Z ~ Normal(10,1)
f = Factor("x0x1f1", "LinearRelative", ["x0", "x1"],
      FactorData(
          fnc=LinearRelative(Normal(10, 0.1)
          ).dump()  # This is a generator for a LinearRelative
      ))
result_id = await addFactor(navability_client, client, f)

Added x1 with result ID 01G002K65X74Z4HS244FPT2RRE


### Visualizing the Variable Probability Belief

The NavAbility WebApp allows visualization (plotting) of the belief state over any of the variable nodes.
Remember the first time executions are slow given the required code compilation, and that future versions of these packages will use more precompilation to reduce first execution running cost.

We first need to trigger a solve for the web application to update its belief estimates. We first make sure that the all tasks have completed:


In [14]:
print(await getStatusLatest(navability_client, result_id))

<StatusMessage(requestId=01G002P9TXCXYY3V39ZX537G9Z, timestamp=2022-04-06 18:37:54.325000, client=<Client(userId=Guest, robotId=SDKpy_0bd, sessionId=Tutorial2_678)>, action=Add:Factor, state=Complete)>



and then trigger the solve. The current belief estimate is ploted in the NavAbility WebApp as soon as the results become available.

In [16]:
solve_request = await solveSession(navability_client, client)

In [15]:
print("https://app.navability.io/cloud/map/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

https://app.navability.io/cloud/map/?userId=Guest&robotStartsWith=SDKpy_0bd&sessionStartsWith=Tutorial2_678


the predicted influence of the `P(Z| X1 - X0) = LinearRelative(Normal(10, 1))` is shown by the `x1` trace.
The predicted belief of `x1` is nothing more than the approximated convolution of the current marginal belief of `x0` with the conditional belief described by `P(Z | X1 - X0)`.


### Mixture Distribution on Next Relative Factor (Wheel Slip)

As the robot continues to drive from `x1` to `x2`, the robot software detects wheel slip has occurred.  Here then is the critical point, how should the next odometry measurement incorporate this ambiguous information.  First, we add the next `ContinuousScalar` variable `x2` as before.  

In [17]:
v = Variable("x2", "IncrementalInference.ContinuousScalar") #VariableType.ContinuousScalar.value)
result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")

Added x2 with result ID 01G002X95YSNKB6VGCYY74VBF2


This time, however, the odometry factor from `x1` uses a more complicated `Mixture` likelihood function.  Given a 40% accuracy in wheel slip detection, we are going to construct a `Mixture` distribution as the measurement, where 40% weight goes to what we think the distance travelled is while the remaining 60% weight is designated to 'faulty' odometry measurement.  

Taking the full encoder turns as 60 units distance, with slip being detected on and off throughout, we estimate the majority slip case as a Rayleigh distribution from 0.  Since we know very little about the non-slip case, let's take the robot distance traveled as uniformly distributed somewhere between 40 up to 60 units. 

In [18]:
mixture_factor = Factor("x1x2f1", "Mixture", ["x1", "x2"],
      FactorData(
            fnc=Mixture(LinearRelative, 
                {
                    "slip": Rayleigh(5), 
                    "noslip": Uniform(35,60)
                },
                [0.4, 0.6],
                2
          ).dump()
      ))
print(mixture_factor.dumps())
result_id = await addFactor(navability_client, client, mixture_factor)

{"label": "x1x2f1", "_version": "0.18.1", "_variableOrderSymbols": ["x1", "x2"], "data": {"eliminated": false, "potentialused": false, "edgeIDs": [], "fnc": {"N": 2, "F_": "PackedLinearRelative", "S": ["slip", "noslip"], "components": [{"_type": "IncrementalInference.PackedRayleigh", "sigma": 5.0}, {"_type": "IncrementalInference.PackedUniform", "a": 35.0, "b": 60.0, "PackedSamplableTypeJSON": "IncrementalInference.PackedUniform"}], "diversity": {"_type": "IncrementalInference.PackedCategorical", "p": [0.4, 0.6]}}, "multihypo": [], "certainhypo": [1, 2], "nullhypo": 0.0, "solveInProgress": 0, "inflation": 5.0}, "tags": ["FACTOR"], "timestamp": "2022-04-06T18:23:16.441Z", "nstime": "0", "fnctype": "Mixture", "solvable": 1}



The `mmo` variable illustrates how a near arbitrary mixture probability distribution can be used as a conditional relationship between variable nodes in the factor graph.
In this case, a 40%/60% balance of a Rayleigh and truncated Uniform distribution which acts as a multi-modal conditional belief.
Interpret carefully what a conditional belief of this nature actually means.

Following the tutorial's practical example frameworks (robot navigation or time travel), this multi-modal belief implies that moving from one of the probable locations in `x1` to a location in `x2` by some processes defined by `mmo=P(Z | X2, X1)` is uncertain to the same 40%/60% ratio.
In practical terms, collapsing (through observation of an event) the probabilistic likelihoods of the transition from `x1` to `x2` may result in the `x2` location being at either 15-20, or 40-65-ish units.
The predicted belief over `x2` is illustrated by plotting the predicted belief after triggering a new solve.

In [19]:
solve_request = await solveSession(navability_client, client)

In [ ]:
# the results updates automatically as the solver finishes. Here is the link to the Geometric map visualizations in th App again:
print("https://app.navability.io/cloud/map/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

Notice the multi-modal belief in the marginal posterior belief of `x3`.  Also note that Kernel Density Estimate methods can appear to produce noisy belief density estimates.  While this does sometimes occur, it is harmless and has no impact on the accuracy of the mean point estimate produced for each mode in the associated belief.


## Adding Variable `x3`

In suspecting that the robot did get stuck and had its wheels slip, we reverse the robot 50 units and add pose variable `x3`,  i.e. a factor measurement `LinearRelative(Normal(-50,1))`

In [20]:
v = Variable("x3", "IncrementalInference.ContinuousScalar") #VariableType.ContinuousScalar.value)
result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")

linear_factor = Factor("x2x3f1", "LinearRelative", ["x2", "x3"],
      FactorData(
          fnc=LinearRelative(Normal(-50, 1)
          ).dump()  # This is a generator for a LinearRelative
      ))
result_id = await addFactor(navability_client, client, linear_factor)

Added x3 with result ID 01G0033J22M5ZC82EYJPGJ0CDY


expands the factor graph to four variables and four factors.
This part of the tutorial shows how a unimodal likelihood (conditional belief) can transmit the bimodal belief currently contained in `x2`.  By solving the graph, we get numerical estimates for the variables:

In [21]:
solve_request = await solveSession(navability_client, client)

and plotting the marginal posterior belief estimates over each variable to see what the position estimates are given available info:

In [22]:
print("https://app.navability.io/cloud/map/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

https://app.navability.io/cloud/map/?userId=Guest&robotStartsWith=SDKpy_0bd&sessionStartsWith=Tutorial2_678


Notice the `x3` trace is a shifted and slightly spread out version of the belief on `x2`, through the convolution with the conditional belief `P(Z | X2, X3)`.

### The Last Factor

Only by introducing more information into the factor graph can inference extract more precise marginal belief estimates for each of the variables.
A final piece of information added to this graph is a factor directly relating `x3` with `x0`.

In [23]:
linear_factor = Factor("x3x0f1", "LinearRelative", ["x3", "x0"],
      FactorData(
          fnc=LinearRelative(Normal(30, 1)
          ).dump()  # This is a generator for a LinearRelative
      ))
result_id = await addFactor(navability_client, client, linear_factor)

print(await getStatusLatest(navability_client, result_id))

Pay close attention to what this last factor means in terms of the probability density traces shown in the previous figure.
The trace for `x3` has two major modes, one that overlaps with `x0, x1` near 0 and a second mode further to the left at -40.
The last factor introduces a shift `LinearRelative(Normal(40,1))` which essentially aligns the left most mode of `x3` back onto `x0`.

This last factor forces a mode selection through consensus.
By doing global inference, the new information obtained in `x3` will be equally propagated to `x2` where only one of the two modes will remain

## Solve the Graph

Global inference over the full graph can now be triggered.
Again, we use `solveSession` to trigger the final solve in the NavAbility platform.

In [24]:
solve_request = await solveSession(navability_client, client)

In [ ]:
# and the visualization as previously described.
print("https://app.navability.io/cloud/map/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

The resulting posterior marginal beliefs over all the system variables are:

In [28]:
variables = await ls(navability_client, client)
print(f"Variables in graph: {variables}")
ppes = {
        v: (await getVariable(navability_client, client, v)).ppes["default"].suggested
        for v in variables
    }
print(ppes)

Variables in graph: ['x3', 'x2', 'x1', 'x0']
{'x3': [-30.428188344753558], 'x2': [18.881459586223556], 'x1': [10.219333087730702], 'x0': [-0.006425329252770611]}


### The Wheels Did Slip!

Look, the resulting distance from `x1` to `x2` with all the data available turns out to be 10 units -- much less than the 60 units measured by the encoder, coupled with the on and off slip detection.  Our example here is constructed in such a way that when all data is considered together a clear answer can be extracted.  In real situations, the beliefs could be more nuanced -- which is all the more reason to consider non-Gaussian estimation which permanently keeps track of the nuanced features hidden in the data!

## Conclusion

It is important to note that although this tutorial ends with all marginal beliefs having near Gaussian shape and are unimodal, that the package supports multi-modal belief estimates during both the prediction and global inference processes.

## Next Steps

The next tutorial looks at weakly observable (a.k.a. missing / insufficient data, or underdetermined) situations where all Gaussian measurements result in highly non-Gaussian and multi-modal posterior estimates.

### Case Study: Marine Surface Navigation

Furthermore, this tutorial has practical value.  For example, NavAbility developed a GPS-denied navigation technique for marine surface vehicles whereby consecutive radar sweeps are correlated for extracting odometry measurements.  These correlations, as it turns out, are non-Gaussian and on many occasions and exhibit multi-modal behavior similar to the tutorial illustrated above.

**Real World Non-Gaussian Odometry Measurement**

The left image below shows two consecutive `360 degree` radar sweeps from a marine surface vehicle.  The right-hand image shows a slice from the dense correlation map on the `SpecialEuclidean(2)` manifold when looking for the best alignment between the two radar sweeps -- notice the non-Gaussian / multi-modal behavior!  This type of non-Gaussian measurement can readily be used in the NavAbility and Caesar.jl solver system.  See the [NavAbility Marine Surface Navigation Case Study page](https://www.navability.io/applications/marine/) for more details.

![img](http://www.navability.io/wp-content/uploads/2022/04/MarineRadarAlignFigure-1024x485-1.png)